In [ ]:
# install yolo and add custom YAML file
!git clone https://github.com/ultralytics/yolov5
!pip install -U -r yolov5/requirements.txt

In [1]:
#@title __Initialize Notebook__

%reload_ext autoreload
%autoreload 2
!rm -rf sample_data

import os
import cv2
import torch
import urllib
import numpy as np
import pandas as pd
from PIL import Image
import ipywidgets as widgets
from tqdm.autonotebook import tqdm
from google.colab.widgets import Grid
from google.colab.patches import cv2_imshow
from IPython.display import HTML, display, clear_output


# define path constants
DATASET_IMAGES_PATH = '/content/drive/MyDrive/MOT Project/dataset/images.zip' #@param {type:"string"}
DATASET_LABELS_PATH = '/content/drive/MyDrive/MOT Project/dataset/labels_militaryVehicle.zip' #@param {type:"string"}
TEST_VIDEOS_PATH = '/content/drive/MyDrive/test videos.zip' #@param {type:"string"}
YOLO_YAML_PATH = '/content/drive/MyDrive/MOT Project/dataset/customDataset_militaryVehicle.yaml' #@param {type:"string"}

# add YAML file to YOLO
%cp "$YOLO_YAML_PATH" "/content/yolov5/data"


clear_output()

In [2]:
#@title __Setup__

class Button:
    def __init__(self, description, function, function_args=None):
        self.description = description
        self.function = function
        self.function_args = function_args

        self.button = widgets.Button(description=self.description)
        self.output = widgets.Output()

        def on_button_clicked(b):
            with self.output:
                if function_args:
                    self.function(*self.function_args)
                else:
                    self.function()
            
        self.button.on_click(on_button_clicked)

    def display(self):
        display(self.button, self.output)


def runtime_details_block():
    def button_func():
        print(f'torch {torch.__version__}')
        if torch.cuda.is_available():
            gpu_data = torch.cuda.get_device_properties(0)
            gpu_name = gpu_data.name
            gpu_mem  = f'{gpu_data.total_memory*1e-9:.02f} Gb'
            print(f'GPU: {gpu_name}\nMemory: {gpu_mem}')
        else:
            print('CPU')

    button = Button(description="Show Runtime Details", function=button_func)
    button.display()


def get_training_data_button():
    def button_func():
        !mkdir dataset
        %cd dataset
        %cp "$DATASET_IMAGES_PATH" "/content/dataset"
        %cp "$DATASET_LABELS_PATH" "/content/dataset"
        images_zip_name = DATASET_IMAGES_PATH.split('/')[-1]
        labels_zip_name = DATASET_LABELS_PATH.split('/')[-1]
        !unzip "$images_zip_name"; rm "$images_zip_name"
        !unzip "$labels_zip_name"; rm "$labels_zip_name"
        !rm -rf __MACOSX/
        %cd ../

        clear_output()
        print('Loaded training data')

    button = Button(description="Get train dataset", function=button_func)
    button.display()

def get_test_videos_button():
    def button_func():
        !mkdir test_videos
        %cd test_videos
        %cp "$TEST_VIDEOS_PATH" "/content/test_videos"
        zip_name = TEST_VIDEOS_PATH.split('/')[-1]
        !unzip "$zip_name"; rm "$zip_name"; rm -rf __MACOSX/
        %cd ../

        clear_output()
        print('Loaded test videos')

    button = Button(description="Get test videos", function=button_func)
    button.display()


################################################################################
#                                Create Grid
################################################################################

grid = Grid(1, 3)
for i, _ in enumerate(grid):
    if i==0:
        runtime_details_block()
    if i==1:
        get_training_data_button()
    if i==2:
        get_test_videos_button()

,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Show Runtime Details', style=ButtonStyle())

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Get test videos', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>

In [3]:
%cd /content/yolov5

/content/yolov5


# Train Yolo

In [4]:
#@title __Training Loop__

# define path constants
img_size = 640 #@param {type:"integer"}
batch_size = 64 #@param {type:"integer"}
epochs = 20 #@param {type:"integer"}
yaml_name = "customDataset_militaryVehicle.yaml" #@param {type:"string"}
weights = "yolov5s.pt" #@param {type:"string"}
model_name = "one_class_YOLOV5s" #@param {type:"string"}


!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --cache --data {yaml_name} --weights {weights} --name {model_name} --cache

train: weights=yolov5s.pt, cfg=, data=customDataset_militaryVehicle.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=one_class_YOLOV5s, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-375-gd1182c4 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, trans

In [ ]:
# example of zipping the run results, so it easy do download and save for later
!zip -r yolov5l_person_military_seperate.zip /content/yolov5/runs/train/yolov5l_person_military_seperate/

In [ ]:
#@title __Check Detections on a Video__

video_path = "../test_videos/Test 1.mp4" #@param {type:"string"}
weights_path = "/content/yolov5/runs/train/yolov5l_person_military_seperate/weights/best.pt" #@param {type:"string"}
result_name = "yolov5l_person_military_seperate_Test1" #@param {type:"string"}
confidence = 0.35 #@param {type:"slider", min:0, max:1, step:0.05}

!python detect.py --source {video_path} --weights {weights_path} --conf {confidence} --name {result_name}